# Protocol: Maximum Size Hash table

Date: 16.09.2021

## Question

Does increasing the Grapple hash table size to full 48 KB (shared memory maximum size) increase Waypoint Discovery in relation to the executed VTs such that it is similar to the results from the Grapple paper?

## Hypothesis

As the hash table capacity significantly affects the state space coverage, having the same hash table capacity as in the paper allows reproduction of their results.

## Setup

- GPU: NVIDIA GeForce RTX 2080 Ti
- Program: `main` branch, commit e160572 and `feature/EXP-02-max-size-hash-table` branch, commit c9cd0ec
- Model: Waypoints model
- CUDA_FLAGS: `-DGRAPPLE_MODEL=WaypointsState`

## Implementation

For each experiment, we execute 80 runs à 250 VTs (= 20000 VTs).

First, we run our implementation (`main` branch) that can store $2^{18} = 262144$ states in its hash table.

```
$ time ./build/grapple -s 1736331306 -n 80
run,block,thread,state,uniques,visited,visited_percent,vts,total_visited
...
79,,,,,4.74347e+08,11.0443,20000,63568607

real    0m35.464s
user    0m35.198s
sys     0m0.184s
```

Full output data is available at [EXP-02-large-hash-table-1.csv](./data/EXP-02-large-hash-table-1.csv).

---

Then, we run the maximum-size hash table (`feature/max-size-hash-table` branch) that can store $393056$ states in its hash table.

```
$ time ./build/grapple -s 1736331306 -n 80
run,block,thread,state,uniques,visited,visited_percent,vts,total_visited
...
79,,,,,9.9405e+08,23.1445,20000,91937680

real    0m50.373s
user    0m50.098s
sys     0m0.208s
```

Full output data is available at [EXP-02-large-hash-table-2.csv](./data/EXP-02-large-hash-table-2.csv).

## Evaluation

The larger hash table (2nd experiment) yields 2.1x as many visited states. ($23/11 \approx 2.1$)

The larger hash table in total only visits 1.45x as many states as the smaller one. ($91937680/63568607 \approx 1.45$)

In the paper, a Grapple Swarm Verification with 32 threads (called *Full-Warp*) and 392800 hash table slots per VT discovers 85 Waypoints, equalling 85% state space coverage, after 20000 VTs. This is about 3.7x as many as our large-size hash table implementation, which only explores 23% of the state space in 20000 VTs. (column `visited_percent` in the output above)

## Conclusion, Discussion

On this particular model, a larger hash table seems beneficial, as the state space coverage can be more than doubled visiting only 1.45x the states.

Even though our large-size hashtable is slightly larger than the one supposedly used in the paper, we are far from reproducing its results.

Thus, our hypothesis cannot be confirmed.